In [0]:
import warc
import urllib2
import gzip
import json
import requests
import io
from io import StringIO
from bs4 import BeautifulSoup
import tqdm
from tqdm import tqdm
import numpy as np

In [0]:
def search_domain(domain):

  record_list = []
  print "[*] Trying target domain: %s" % domain
  
  for index in index_list:
      
      #print "[*] Trying index %s" % index
      
      cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
      cc_url += "url=%s&matchType=domain&output=json" % domain
      
      response = requests.get(cc_url)
      
      if response.status_code == 200:
          
          records = response.content.splitlines()
          
          for record in records:
              record_list.append(json.loads(record))

  return record_list        


def index_url(site,keyword):
  index_name = 'https://index.commoncrawl.org/CC-MAIN-2019-13-index?url='+site+'%2F'+keyword+'&output=json' # star symbol ivalli
  resp = requests.get(index_name)
  #print(resp)
  return resp

def pages(response):
  try:
    from cStringIO import StringIO
  except:
    from StringIO import StringIO
  pages = [json.loads(x) for x in response.content.strip().split('\n')]
  #print('length of pages:',len(pages))
  data_whole =[]
  data_http  =[]
  for count in range(0,len(pages)):
    if count <= 50 :
      page =pages[count]
      offset, length = int(page['offset']), int(page['length'])
      offset_end = offset + length - 1
      prefix = 'https://commoncrawl.s3.amazonaws.com/'
      resp = requests.get(prefix + page['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
      raw_data = StringIO(resp.content)
      f = gzip.GzipFile(fileobj=raw_data)
      data = f.read()
      dd,links,http_link = beautifulsoup_ccw(data)
      data_whole.append(np.unique(links))
      data_http.append(http_link)
    else:
      break;
  return data,dd,links,data_whole,data_http

def beautifulsoup_ccw(data):
  links =[]
  http_links =[]
  soup = BeautifulSoup(data)
  dd = soup.find_all('a')
  for link in soup.find_all('a'):#,class_ = 'realStory'):
    if link.has_attr('href'): # print(link)
      if('http' in link['href']):
        #print('')
        http_links.append(link['href']) 
      else:
        if link['href'] not in link :
          sub_index = link['href'].find('story')
          if sub_index != -1:
            print(link['href'])
            links.append(link['href'])
          else:
            pass
        else:
          pass
    else:
      break;
  return dd,links,http_links

def beautifulsoup_urlhit(prefix,all_links)
  from tqdm import tqdm
  paragraph =[]
  for count in range(0,len(all_links)):
    for i in all_links[count]:
      link = i
      quote_page = prefix+link
      from urllib2 import Request, urlopen, URLError
      req =urllib2.Request(quote_page)
      try:
        response = urlopen(req)
      except URLError as e:
        if hasattr(e, 'reason'):
          print 'We failed to reach a server.'
          print 'Reason: ', e.reason
        elif hasattr(e, 'code'):
          print 'The server couldn\'t fulfill the request.'
          print 'Error code: ', e.code
      else:
        soup = BeautifulSoup(response, 'html.parser')
        find = soup.find_all('p')
        for text in find:
          f= open("commoncrawl_soccer.txt","a+")
          f.write(text.get_text().encode('utf-8')) 
          f.write('\n')

In [0]:
index_list= ["2019-13","2019-09","2019-04"]

In [0]:
resp_s = index_url('espn.com','soccer*')
data_s,bs_s,links_s,all_links,http_link = pages(resp_s)            

In [0]:
prefix = 'http://www.espn.com'
urls = all_links
beautifulsoup_urlhit(prefix,urls)